In [1]:
# to tackle next:
# write scrappers for other websites (booking next priority) to compare descriptions
# then apply use/test case with one or two cities (one Luxembourg) for all four websites and compare descriptions --> requires rudimentary comparison algorithm
# when scrappers for all websites running improve some details like:
# write function testing the city links before def init? would have to put all the crawling in extra function?
# have all the print statements be exported in txt for each city as a log which then could be searched?
# descriptions sometimes not automatically translated but put in original language and translated to english in one description block --> how solve only getting english one? ignore?
# make crawler run several times, always adding the rooms not yet in the data
# write function within class to create a dataset from the object characteristics
# crawl other things like title, more info, pictures (pictures cannot be part of dataset but maybe exported seperately into predefined data folder structure)

In [2]:
#improvements to do:
# stemming room type (singular) --> do based on if ends with s --> SOLVED
# address can also be 5 long --> with floor --> SOLVED
# sometimes the floor is in front of the street name without comma seperation! --> need to check, fixable? --> SOLVED
# sometimes extra text in address? --> cannot be fixed --> NOISE
# address can have unnnecessary commas --> clean up --> SOLVED
# somehow get encoding right for special signs --> SOLVED
# sometimes comma after street number as well (then it can be 4 instead of 3 long or 5 instead of 4) --> need to check the structure of the address --> SOLVED
# what if apartement type is two words? --> when get_text, select comma as seperator, then split with comma, only keep first to instances [0:2] --> SOLVED
# trailing spaces at the beginning of the description --> SOLVED
# seperate street number? --> could split street, go through with try and except to convert to int --> problem if we have extra text, entity recognition which is not a number for street?
# if postal code more than 2 because letter in front of postal code --> just piCk the last two instances in the list --> SOLVED

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import emoji
import re
import pytz
from requests_html import HTMLSession
import spacy
import pandas

/home/fbohm/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
nlp = spacy.load("en_core_web_trf")
city_list = ['lu/luxembourg', 'de/munich', 'fr/paris', 'es/barcelona']
main_url = 'https://www.booking.com'

In [5]:
class booking_data:
    def __init__ (self, city_list, main_url):
        #set up basics
        self.cities_url = {}
        self.number_properties = {}
        self.extraction_date = {}
        self.rooms = {}
        self.description = {}
        self.host = {}
        self.address = {}
        self.street = {}
        self.district = {}
        self.postal_code = {}
        self.city = {}
        self.country = {}
        self.title = {}
        self.type = {}
        self.index = {}
        self.platform = {}
        
        for city in city_list:
            url = f'{main_url}/apartments/city/{city}.html'
            
            #get the url for the page with all flats from the start page
            session = HTMLSession()
            request = session.get(url)
            soup = BeautifulSoup(request.content, 'html.parser')
            for a in soup.find_all(href=True):
                if a['href'].find('searchresults')!=-1 and a['href'].find('theme_landing_city')!=-1:
                    show_all = a['href']
                    break
            final_url = main_url + show_all
            
            self.cities_url[city] = [final_url]
            self.number_properties[city] = []
            self.extraction_date[city] = []
            self.rooms[city] = []
            self.description[city] = []
            self.host[city] = []
            self.address[city] = []
            self.street[city] = []
            self.district[city] = []
            self.postal_code[city] = []
            self.city[city] = []
            self.country[city] = []
            self.title[city] = []
            self.type[city] = []
            self.index[city] = []
            self.platform[city] = []
            
        print('finished basic setup\n')
    
    def DataCollection (self, city_list, log_name):
        log = open(log_name, 'a')
        session = HTMLSession()
        for city in self.cities_url:
            
            url = self.cities_url[city][0]
            request = session.get(url)
            soup = BeautifulSoup(request.content, 'html.parser')
            amount_prop = soup.find(class_ = 'e1f827110f d3a14d00da')
            amount = amount_prop.get_text(strip = True).split()
            self.number_properties[city].append(int(amount[1]))
            
            self.extraction_date[city].append(new_date)            
            
            log.write('\nworking on city: ' + city + '\n')
            print('working on city: ' + city + '\n')
            pagecounter = 0
            totalcounter = 0
            desccounter = 0
            start = time.time()
            offset = 25
            #access pages and crawl room links
            start_pages = time.time()
            for site in self.cities_url[city]:
                start_page = time.time()
                log.write('\ntrying to extract room links from: ' + site + '\n')
                sub = '/hotel'
                request = session.get(site)
                soup = BeautifulSoup(request.content, 'html.parser')
                links = []
                for a in soup.find_all('a', href=True):
                    link = a['href']
                    res = link.find(sub)
                    if res != -1:
                        cut = link.find('html') + 4
                        final = link[:cut]
                        if link.find('index') == -1 and final not in self.rooms[city] and final not in links:
                            links.append(final)
                if len(links) > 0:
                    pagecounter += 1
                    for element in links:
                        self.rooms[city].append(element)
                    totalcounter += len(links)
                    log.write('\tadded a total of ' + str(len(links)) + ' room URLs\n')
                    log.write('\tcrawled a total of ' + str(pagecounter) + ' pages so far\n')
                    self.cities_url[city].append(url + '&offset=' + str(offset))
                    offset += 25
                else:
                    del self.cities_url[city][-1]
                    log.write('\tabove page did not yield any new room links, meaning we have reached the last page\n')
                    log.write('\nreached the final page at ' + str(pagecounter) + ' and added a total of ' + str(totalcounter) + ' room URLs\n')
                    
                log.write('\tdone with extracting room URLs within ' + str(time.time()-start_page) + ' seconds\n')
    
            log.write('done with all page URLs within ' + str(time.time()-start_pages) + ' seconds\n')
            log.write('\twe expected a total of ' + str(self.number_properties[city][0]) + ' URLs\n')
            if self.number_properties[city][0] - totalcounter > 0:
                log.write('\twe have crawled less room URLs than expected with a difference of: ' + str(self.number_properties[city][0] - totalcounter) + '\n')
            elif self.number_properties[city][0] - totalcounter == 0:
                log.write('\twe have no difference between the expected and the actually crawled room URLs\n')
            elif self.number_properties[city][0] - totalcounter < 0:
                log.write('\twe crawled more room URLs than expected with a difference of: ' + str(totalcounter - self.number_properties[city][0]) + '\n')
            
            #extract room details
            descrawl_time = time.time()
            for room in self.rooms[city]:
                
                room_time = time.time()
                log.write('\ndoing accommodation: ' + room + '\n')
                request = session.get(room)
                soup = BeautifulSoup(request.content, 'html.parser')
                
                self.index[city].append(desccounter)
                self.platform[city].append(11)
                
                log.write('\tdoing accommodation description\n')
                desc = soup.find(id='property_description_content')
                desc_all = ''
                for p in desc:
                    text = p.get_text(' ', strip = True)
                    text_low = text.lower()
                    if "genius discount" not in text_low:
                        desc_all += text_low + ' '
                    desc_clean = desc_all.strip(' ')
                self.description[city].append(desc_clean)
                log.write('\taccommodation description done\n')
                
                log.write('\tdoing accommodation host\n')
                host = soup.find(class_='e1f827110f')
                if host is None:
                    self.host[city].append('NaN')
                else:
                    doc = nlp(host.get_text())
                    if doc.ents != ():
                        ent = doc.ents[0].text
                        self.host[city].append(ent.lower())
                    else:
                        self.host[city].append('NaN')
                log.write('\taccommodation host done\n')  
                
                log.write('\tdoing accommodation title and type\n')
                title = soup.find(class_='hp__hotel-title')
                title_ = title.get_text(',', strip = True).lower()
                title_text = title_.split(',')
                room_type = title_text[0]
                if room_type[-1] == 's':
                    room_type = room_type[0:-1]
                room_title = title_text[1]
                self.title[city].append(room_title)
                self.type[city].append(room_type)    
                log.write('\taccommodation title and type done\n')
                
                log.write('\tdoing accommodation address\n')
                address = soup.find(class_='hp_address_subtitle js-hp_address_subtitle jq_tooltip')
                address_ = address.get_text(strip = True).replace(', ', ',').lower()
                address_clean = re.sub(',+', ',', address_)
                self.address[city].append(address_clean)
                address_text = address_clean.split(',')
                if len(address_text) == 3:
                    self.street[city].append(address_text[0])
                    self.district[city].append('NaN')
                    self.country[city].append(address_text[2])
                    city_postcode = address_text[1].split()
                    self.postal_code[city].append(city_postcode[-2])
                    self.city[city].append(city_postcode[-1])
                
                elif len(address_text) == 4: #need to check if first instance is just a number (length)
                    if len(address_text[0]) == 1: #technically the situation of having length 3
                        street_reco = address_text[0] + ' ' + address_text[1]
                        self.street[city].append(street_reco)
                        self.district[city].append('NaN')
                        self.country[city].append(address_text[3])
                        city_postcode = address_text[2].split()
                        self.postal_code[city].append(city_postcode[-2])
                        self.city[city].append(city_postcode[-1])
                    else:
                        self.street[city].append(address_text[0])
                        self.district[city].append(address_text[1])
                        self.country[city].append(address_text[3])
                        city_postcode = address_text[2].split()
                        self.postal_code[city].append(city_postcode[-2])
                        self.city[city].append(city_postcode[-1])
                            
                elif len(address_text) == 5: #for floor included, need to check if first instance is number
                    if len(address_text[0]) == 1: #technically the situation of having length 4
                        street_reco = address_text[0] + ' ' + address_text[1]
                        self.street[city].append(street_reco)
                        self.district[city].append(address_text[2])
                        self.country[city].append(address_text[4])
                        city_postcode = address_text[3].split()
                        self.postal_code[city].append(city_postcode[-2])
                        self.city[city].append(city_postcode[-1])

                    else:
                        self.street[city].append(address_text[1])
                        self.district[city].append(address_text[2])
                        self.country[city].append(address_text[4])
                        city_postcode = address_text[3].split()
                        self.postal_code[city].append(city_postcode[-2])
                        self.city[city].append(city_postcode[-1])
                else:
                    self.street[city].append('error during extraction')
                    self.district[city].append('error during extraction')
                    self.postal_code[city].append('error during extraction')
                    self.city[city].append('error during extraction')
                    self.country[city].append('error during extraction')
                log.write('\taccommodation address done\n')
                
                desccounter += 1
                log.write('\tadded description for room within ' + str(time.time()-room_time) + 'seconds\n')
                log.write('\tdescriptions scrapped: ' + str(desccounter) + '\n')
                
            log.write('\ndone scrapping room descriptions with a total of ' + str(desccounter) + ' within ' + str(time.time()-descrawl_time) + ' seconds\n')
        
            log.write('done with city: ' + city + ' within ' + str((time.time()-start)/60) + ' minutes\n')

            print('done with city: ' + city + ' within ' + str((time.time()-start)/60) + ' minutes\n')
            print('compare the lengths of the relevant data dictionaries, they should be identical:')
            print('\tlength of self.rooms', len(self.rooms[city]))
            print('\tlength of self.description', len(self.description[city]))
            print('\tlength of self.host', len(self.host[city]))
            print('\tlength of self.address', len(self.address[city]))
            print('\tlength of self.street', len(self.street[city]))
            print('\tlength of self.district', len(self.district[city]))
            print('\tlength of self.postal_code', len(self.postal_code[city]))
            print('\tlength of self.city', len(self.city[city]))
            print('\tlength of self.country', len(self.country[city]))
            print('\tlength of self.title', len(self.title[city]))
            print('\tlength of self.type', len(self.type[city]))

            print('\nand for further checks, the other to dictionaries with the metadata:')
            print('\tpage links:', self.cities_url)
            print('\tproperty numbers:', self.number_properties)
            print('\textraction date:', self.extraction_date)
        
        log.close()
        
    def dataset (self, csv_name):
        df_dic = {'index': [], 'platform': [], 'room': [], 'title': [], 'type': [], 'desc': [], 'host': [], 'address': [], 
                'street': [], 'district': [], 'postal_code': [], 'city': [], 'country': []}
        for city in city_list:
            for element in self.rooms[city]:
                df_dic['room'].append(element)
            for element in self.description[city]:
                df_dic['desc'].append(element)
            for element in self.host[city]:
                df_dic['host'].append(element)
            for element in self.address[city]:
                df_dic['address'].append(element)
            for element in self.street[city]:
                df_dic['street'].append(element)
            for element in self.district[city]:
                df_dic['district'].append(element)
            for element in self.postal_code[city]:
                df_dic['postal_code'].append(element)
            for element in self.city[city]:
                df_dic['city'].append(element)
            for element in self.country[city]:
                df_dic['country'].append(element)
            for element in self.title[city]:
                df_dic['title'].append(element)
            for element in self.type[city]:
                df_dic['type'].append(element)
            for element in self.index[city]:
                df_dic['index'].append(element)
            for element in self.platform[city]:
                df_dic['platform'].append(element)

        df = pandas.DataFrame.from_dict(df_dic)
        df.to_csv(csv_name, index = False, encoding = 'utf-8')
        return df
            

In [6]:
date = datetime.now(pytz.timezone('Europe/Berlin'))
new_date = f'{date.day}-{date.month}-{date.year}_{date.hour}:{date.minute}'
obj = booking_data(city_list,main_url)

finished basic setup



In [6]:
obj.DataCollection(city_list, 'log_booking_' + str(new_date) + '.txt')

/home/fbohm/.local/lib/python3.7/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


compare the lengths of the relevant data dictionaries, they should be identical:
	length of self.rooms 134
	length of self.description 134
	length of self.host 134
	length of self.address 134
	length of self.street 134
	length of self.district 134
	length of self.postal_code 134
	length of self.city 134
	length of self.country 134
	length of self.title 134
	length of self.type 134

and for further checks, the other to dictionaries with the metadata:
	page links: {'lu/luxembourg': ['https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1FCAMY9gMoiQFCCmx1eGVtYm91cmdIM1gEaGmIAQGYAQm4ARjIAQ3YAQHoAQH4AQuIAgGoAgS4Asyhu5YGwAIB0gIkYzRkNWE1OGMtZTA1Yi00YTZkLThhNDYtY2UxOGY0ZjNhZjdh2AIG4AIB&sid=7d6415a7a9ca5af04855664038e25183&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city', 'https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1FCAMY9gMoiQFCCmx1eGVtYm91cmdIM1gEaGmIAQGYAQm4ARjIAQ3YAQHoAQH4AQuIAgGoAgS4Asyhu5YGwAIB0gIkYzRkNWE1OGMtZTA1Yi00YTZkLThhNDYtY2UxOGY0ZjNh

In [7]:
booking_df = obj.dataset('booking_' + str(new_date) + '.csv')

### **Code dump and tries**

In [125]:
city = 'lu/luxembourg'
url = f'{main_url}/apartments/city/{city}.html'
print(url)

https://www.booking.com/apartments/city/lu/luxembourg.html


In [126]:
url.find('searchresults')

-1

In [127]:
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
show_all = soup.find_all(class_='bui-button bui-button--wide bui-button--primary')

In [128]:
for a in soup.find_all(href=True):
    if a['href'].find('searchresults')!=-1 and a['href'].find('theme_landing_city')!=-1:
        print ("Found the URL:", a['href'])

Found the URL: /searchresults.html&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city
Found the URL: /searchresults.html&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city


In [129]:
print(show_all[0]['href'])

/searchresults.html&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city


In [130]:
final = main_url+show_all[0]['href']

In [131]:
print(final)

https://www.booking.com/searchresults.html&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city


In [132]:
test = final + '&prefer_site_type=mdot'
print(test)

https://www.booking.com/searchresults.html&city=-1736191;nflt=sth%253D1;pptfromtl=1;src=theme_landing_city&prefer_site_type=mdot


In [133]:
#try with html request package

In [134]:
session = HTMLSession()
request = session.get(final)
soup_page = BeautifulSoup(request.content, 'html.parser')

In [ ]:
#<h1 class="e1f827110f d3a14d00da">Munich: 562 properties found</h1>

In [135]:
info = soup_page.find(class_ = 'e1f827110f d3a14d00da')
print(info)

<h1 class="e1f827110f d3a14d00da">Luxembourg: 135 properties found</h1>


In [138]:
info.get_text(strip = True).split()

['Luxembourg:', '135', 'properties', 'found']

In [19]:
for a in soup_page.find_all(href=True):
    print ("Found the URL:", a['href'])

Found the URL: https://t-cf.bstatic.com
Found the URL: https://t-cf.bstatic.com
Found the URL: https://shelves.booking.com/
Found the URL: https://t-cf.bstatic.com/static/css/main_cft.iq_ltr/0c158ed9ab4ab2b83e0d772a9391a926b7d81b8c.css
Found the URL: https://t-cf.bstatic.com/static/css/main_exps_cft.iq_ltr/a663a9ffb0ea47e7b61e07a4d61229810d16897e.css
Found the URL: https://t-cf.bstatic.com/static/css/gprof_icons_cft.iq_ltr/5ba8ce5863bb52d648f365182805b8c4775d1281.css
Found the URL: https://t-cf.bstatic.com/static/css/searchresults_cft.iq_ltr/f468a1bbced4ca77a54df9bbad8a9b0ed871e642.css
Found the URL: https://t-cf.bstatic.com/static/css/hotel_base_cft.iq_ltr/dee298b55ffe0ead937165fd57b47462c1a68e73.css
Found the URL: https://t-cf.bstatic.com/static/css/hotel_experiments_cft.iq_ltr/749487e170e30b51a0468773b3d522202863f487.css
Found the URL: https://t-cf.bstatic.com/static/css/index_cft.iq_ltr/373a51925132f84862f604ab53702002f0988a16.css
Found the URL: https://t-cf.bstatic.com/static/css/

In [20]:
sub = '/hotel'
links = []
for a in soup_page.find_all('a', href=True):
    link = a['href']
    res = link.find(sub)
    if res != -1:
        cut = link.find('html') + 4
        final = link[:cut]
        if link.find('index') == -1 and final not in links:
            links.append(final)

In [21]:
print(links)

['https://www.booking.com/hotel/lu/spacious-flat-in-the-heart-of-the-city-center-ideal-for-a-family.en-gb.html', 'https://www.booking.com/hotel/lu/cab-ap-39-art.en-gb.html', 'https://www.booking.com/hotel/lu/modern-100m2-flat-2mn-walking-from-train-station-in-city-center.en-gb.html', 'https://www.booking.com/hotel/lu/guillaume-suites.en-gb.html', 'https://www.booking.com/hotel/lu/the-central.en-gb.html', 'https://www.booking.com/hotel/lu/smartflats-design-luxembourg.en-gb.html', 'https://www.booking.com/hotel/lu/urban-apartment-in-the-nightlife-area.en-gb.html', 'https://www.booking.com/hotel/lu/lago-welcome-clausen.en-gb.html', 'https://www.booking.com/hotel/lu/belair-city-centre-luxury-apartment-2bedrooms-luxembourg.en-gb.html', 'https://www.booking.com/hotel/lu/perfect-for-solo-traveler-15min-walk-to-lux-city-center.en-gb.html', 'https://www.booking.com/hotel/lu/chic-city-center-flat.en-gb.html', 'https://www.booking.com/hotel/lu/the-luxury-green-penthouse-in-city-center-with-terrac

In [22]:
print(len(links))

25


In [23]:
test2 = 'https://www.booking.com/searchresults.html&city=-1736191&nflt=sth%3D1&pptfromtl=1&src=theme_landing_city&ifl=1&offset=25'

In [24]:
session = HTMLSession()
request = session.get(test2)
soup_page = BeautifulSoup(request.content, 'html.parser')

In [25]:
sub = '/hotel'
links2 = []
for a in soup_page.find_all('a', href=True):
    link = a['href']
    res = link.find(sub)
    if res != -1:
        cut = link.find('html') + 4
        final = link[:cut]
        if link.find('index') == -1 and final not in links2:
            links2.append(final)

In [26]:
print(links2)

['https://www.booking.com/hotel/lu/art-deco-100m2-flat-2-min-walking-from-train-station-in-city-center.en-gb.html', 'https://www.booking.com/hotel/lu/spacious-modern-flat-in-the-heart-of-the-city-center.en-gb.html', 'https://www.booking.com/hotel/lu/beautiful-wooded-2br-open-space-apartment.en-gb.html', 'https://www.booking.com/hotel/lu/new-and-luxury-2-bedrooms-parking-place-included.en-gb.html', 'https://www.booking.com/hotel/lu/bright-chic-1br-apt-w-balcony-and-parking-close-to-cloche-d-or-shopping-center.en-gb.html', 'https://www.booking.com/hotel/lu/luminous-penthouse-with-private-terrace-free-parking.en-gb.html', 'https://www.booking.com/hotel/lu/white-house.en-gb.html', 'https://www.booking.com/hotel/lu/lux-city-hamilius-modern-amp-spacious-apart-w-view-luxembourg.en-gb.html', 'https://www.booking.com/hotel/lu/luxembourg-city-appartement-105m2-with-balcony.en-gb.html', 'https://www.booking.com/hotel/lu/modern-100m2-flat-2mn-walking-from-train-station-in-city-center.en-gb.html', 

In [27]:
print(len(links2))

25


In [28]:
for link in links:
    if link in links2:
        print(link)

https://www.booking.com/hotel/lu/modern-100m2-flat-2mn-walking-from-train-station-in-city-center.en-gb.html


In [29]:
#result: with the html session it works!, can go through the pages and get the room links, then work cut them appropriately for beauty and add them to list
#to access the next page can add the offset (starting with 25 and increasing by 25) manually until the returned hotel links list is empty (meaning pages does not exist)

In [30]:
#now: how access the text?
#try a more greedy approach, try to filter the text, try not to rely on div classes
#get the text but not sorted or interpretable --> get div classes displayed
#how to seperate text in the greedy approach?

In [246]:
#room1 = links[0]
#room2 = links2[0]
room3 = 'https://www.booking.com/hotel/lu/spacious-flat-in-the-heart-of-the-city-center-ideal-for-a-family.en-gb.html'
room4 = 'https://www.booking.com/hotel/lu/cab-ap-39-art.en-gb.html'
room5 = 'https://www.booking.com/hotel/lu/luminous-4th-floor-studio-in-lux-belair.en-gb.html'
room6 = 'https://www.booking.com/hotel/lu/cactus-flat-70m3.en-gb.html'
room7 = 'https://www.booking.com/hotel/lu/studio-house.en-gb.html'
room8 = 'https://www.booking.com/hotel/lu/studio-chez-tony.en-gb.html'

In [247]:
session = HTMLSession()
request = session.get(room8)
soup_room = BeautifulSoup(request.content, 'html.parser')

In [146]:
info = soup_room.find_all(text = True)

In [148]:
print(info)

['html', '\n', "\nYou know you could be getting paid to poke around in our code?\nWe're hiring designers and developers to work in Amsterdam:\nhttps://careers.booking.com/\n", '\n', ' wdot-802 ', '\n', '\ndocument.addEventListener(\'DOMContentLoaded\', function () {\n//c360 javascript tracker first iteration\n//sends a track request to c360 http tracker\n//in order to use it:\n//1. inline the c360Tracker.js in the page you need to use it\n//2. in your js file:\n//\n// var c360Tracker = B.require(\'c360Tracker\');\n// var event = {\n// action_name:"accommodation_checkout_confirmation_viewed",\n// action_version :"0.2.0",\n// content : { "transaction_id" : 123434},\n// user : { "BKNG_user_id": 123434}\n// };\n// c360Tracker.track(event);\nB.define(\'c360Tracker\', function () {\nvar enrichedContext = {};\nvar configuration = {\nvalidateInput: false\n};\nvar track = function (event) {\nif (event == null) {\nreturn "event object is null or empty";\n} else {\n//if (enrichedContext.userID !=

In [33]:
for a in soup_room.find_all(href=True):
    if 'bstatic' in a['href']:
        print ("Found the URL:", a['href'])

Found the URL: https://cf.bstatic.com
Found the URL: https://cf.bstatic.com
Found the URL: https://cf.bstatic.com/static/css/main_cloudfront_sd.iq_ltr/0e98cf1a11514412f871e86eb1e315d0a68b06ee.css
Found the URL: https://cf.bstatic.com/static/css/main_exps_cloudfront_sd.iq_ltr/aaa257c9c753bd68dbcc81ca7ce75036d7f5dd0d.css
Found the URL: https://cf.bstatic.com/static/css/gprof_icons_cloudfront_sd.iq_ltr/004bc18db1a7325e3d00673f69f26e0dbd5d300e.css
Found the URL: https://cf.bstatic.com/static/css/hotel_base_cloudfront_sd.iq_ltr/abdb3e7b33e2e6dbb9125a7547c57c9164b610e2.css
Found the URL: https://cf.bstatic.com/static/css/hotel_experiments_cloudfront_sd.iq_ltr/7befe36f7725e8634af3e9fcf0e8e44d434cbb16.css
Found the URL: https://cf.bstatic.com/static/css/hotel_experiments_rtrw_cloudfront_sd.iq_ltr/228a38bbac4c448a28f9dc60eeee64891bc8d8fe.css
Found the URL: https://cf.bstatic.com/static/css/searchresults_cloudfront_sd.iq_ltr/95210bf714e6759257715abcbe433eb74c875b8c.css
Found the URL: https://cf.

In [248]:
info = soup_room.find(class_='hp_address_subtitle js-hp_address_subtitle jq_tooltip')

In [267]:
print(info.get_text(strip = True))

3 ETAGE Residence [ JASMIN ] 139 Rue Adolphe Fischer, Gare, 1521 Luxembourg, Luxembourg


In [269]:
j = info.get_text(strip = True).replace(', ', ',').split(',')
print(j)

['3 ETAGE Residence [ JASMIN ] 139 Rue Adolphe Fischer', 'Gare', '1521 Luxembourg', 'Luxembourg']


In [274]:
j[0]

'3 ETAGE Residence [ JASMIN ] 139 Rue Adolphe Fischer'

In [272]:
doc = nlp(j[0])

In [273]:
doc.ents

(139,)

In [ ]:
#structure: floor, street and number, disctrict (optional), postal code and city, country
#or: floor street and number, district (optional), postal code and city, country

In [245]:
for el in doc.ents:
    print(el)

1st
29 Rue du Fort Neipperg
Gare
Luxembourg
Luxembourg


In [244]:
len(doc.ents[1])

5

In [236]:
one = info.get_text(strip = True).replace(', ', ',')

In [258]:
print(one)

1st etage,behind in the building (extra quiet) 29 Rue du Fort Neipperg,Gare,2230 Luxembourg,Luxembourg


In [215]:
inter = re.sub(',+',',',one)

In [255]:
inter

'7, Place de Durbuy, 1428 Luxembourg, Luxembourg'

In [254]:
inter = inter.replace(',', ', ')

In [257]:
doc2 = nlp(inter)
doc2.ents

(7, Place de Durbuy, 1428, Luxembourg)

In [217]:
s = inter.split(',')

In [218]:
print(s)

['7', 'Place de Durbuy', '1428 Luxembourg', 'Luxembourg']


In [222]:
print(len(s[0]))

1


In [60]:
two = '2 rue 1900, Gare, 2157 Luxembourg, Luxembourg'

In [75]:
city = one[2].split()
print(city)
if len(city) == 2:
    print(city[0].lower())
    print(city[1].lower())
else:
    print('error during extraction')
    print(address_text[2].lower())

['2227', 'Luxembourg']
2227
luxembourg


In [64]:
two_split = two.split(',')

In [67]:
two_split[2].split()

['2157', 'Luxembourg']

In [ ]:
#normally address build two ways: street, postal code + city, country or street, district, postal code + city + country

In [95]:
info = soup_room.find(class_='e1f827110f')

In [96]:
nlp = spacy.load("en_core_web_trf")

In [97]:
to_ana = info.get_text()

In [101]:
doc = nlp(info.get_text())

In [102]:
doc.ents != ()

False

In [59]:
ent = doc.ents[0].text

In [60]:
print(ent)

Lofts Inc.


In [36]:
print(info.get_text().split())

['Managed', 'by', 'Arca', 'Properties', 'Luxembourg']


In [190]:
info = soup_room.find(class_='hp__hotel-title')

In [197]:
all_ = info.get_text(',', strip = True).split(',')

In [202]:
print(all_[1])

Pension Classic


In [205]:
only = all_[0:2]

In [206]:
print(only)

['Guest house', 'Pension Classic']


In [143]:
info = soup_room.find(id='property_description_content')

In [144]:
print(info)

<div id="property_description_content">
<p>You're eligible for a Genius discount at Spacious flat in the heart of the City Center! Ideal for a family!! To save at this property, all you have to do is <a class="bui-link" href="https://account.booking.com/auth/oauth2?lang=en-gb&amp;client_id=vO1Kblk7xX9tUn2cpZLS&amp;aid=304142&amp;bkng_action=hotel&amp;redirect_uri=https%3A%2F%2Fsecure.booking.com%2Flogin.html%3Fop%3Doauth_return&amp;dt=1657107299&amp;response_type=code&amp;state=UqkCvVPS3V3ZhP3RGU0i2NoqGj4XLKSfGzs7PDyEewJNJQTMa1x0VBtYUfUJkUChfbe719ILdVToGtvxOM6bbrpbmmFHLSRhGmuR78MlfK1N2nhWEI0gciJLTboNkr33eNwWAQ7uJh9VFNGhYDefDvWD-wJep_9CHLWLJDFE_J4YuOMXu3kycwnvAS8TA3PpWYQsIFz9H7UvNRGBrf-n6hnF1F5iLmxmfwsnuu3BWBONimNt-A5BG4pOvNK75GcEzx-Bf-qIEaEhx559OLiuGvwmx0uwX9V-bhDHsjoUJCHSUIm2DWc_S6ibJLQZ_7N5S2rze7vHv98Kb0rduuLpBoR5P3JjcPTxHWHzvnpULuoWFcIu4xpwUVf7UbtFYEPydn3smLeVv09TtOAa">sign in</a>.</p>
<p>In the Ville Haute district of Luxembourg, close to Contemporary Art Forum Casino Luxembourg, S

In [148]:
info.get_text(' ', strip = True)

"You're eligible for a Genius discount at Spacious flat in the heart of the City Center! Ideal for a family!! To save at this property, all you have to do is sign in . In the Ville Haute district of Luxembourg, close to Contemporary Art Forum Casino Luxembourg, Spacious flat in the heart of the City Center! Ideal for a family! has free WiFi and a washing machine. The property is less than 1 km from Notre Dame Cathedral Luxembourg and a 13-minute walk from Am Tunnel Luxembourg. The apartment features 2 bedrooms, 1 bathroom, bed linen, towels, a flat-screen TV with satellite channels, a dining area, a fully equipped kitchen, and a balcony with city views. Popular points of interest near the apartment include Adolphe Bridge, National Museum of History & Art and Place D'Armes."

In [150]:
text_all = ''
for p in info:
    text = p.get_text(' ', strip = True)
    text_low = text.lower()
    if "genius discount" not in text_low:
        text_all += text_low
print(text_all)

in the ville haute district of luxembourg, close to contemporary art forum casino luxembourg, spacious flat in the heart of the city center! ideal for a family! has free wifi and a washing machine. the property is less than 1 km from notre dame cathedral luxembourg and a 13-minute walk from am tunnel luxembourg.the apartment features 2 bedrooms, 1 bathroom, bed linen, towels, a flat-screen tv with satellite channels, a dining area, a fully equipped kitchen, and a balcony with city views.popular points of interest near the apartment include adolphe bridge, national museum of history & art and place d'armes.


In [153]:
text_all[-1]

'.'

In [152]:
print(fin[-1])

.


In [93]:
info_test = [info.get_text()]

In [94]:
print(info_test)

["\nYou're eligible for a Genius discount at Spacious flat in the heart of the City Center! Ideal for a family!! To save at this property, all you have to do is sign in.\nIn the Ville Haute district of Luxembourg, close to Contemporary Art Forum Casino Luxembourg, Spacious flat in the heart of the City Center! Ideal for a family! has free WiFi and a washing machine. The property is less than 1 km from Notre Dame Cathedral Luxembourg and a 13-minute walk from Am Tunnel Luxembourg.\nThe apartment features 2 bedrooms, 1 bathroom, bed linen, towels, a flat-screen TV with satellite channels, a dining area, a fully equipped kitchen, and a balcony with city views.\nPopular points of interest near the apartment include Adolphe Bridge, National Museum of History & Art and Place D'Armes. \n"]


In [79]:
all_text = []
for a in soup_room.find_all(text = True):
    if a.text != '\n' and a.text != '':
        text = a.text
        text.encode(encoding = 'utf-8')
        text = re.sub('\\n','', text)
        text = re.sub(' +',' ',text)
        text_fin = re.sub('"','',text)
        all_text.append(text_fin)

In [80]:
print(all_text)

['Spacious flat in the heart of the City Center! Ideal for a family!, Luxembourg – Updated 2022 Prices', 'Skip to main content', 'EUR', 'Choose your currency.Your current currency is Euro', 'Choose your language.Your current language is English (UK)', 'Get help with your reservation', 'List your property', 'Register', 'Sign in', 'Stays', 'Flights', 'Flight + Hotel', 'Car rentals', 'Attractions', 'Airport taxis', 'Coronavirus (COVID-19) Support', 'Get the travel advice you need. Read more about possible travel restrictions before you go.', 'Read more', 'Home', 'All apartments', 'Apartments ', 'Spa Hotels', 'Luxembourg', 'Apartments ', 'Hotels', 'Luxembourg', 'Apartments ', 'Hotels', 'Car hire', 'Spacious flat in the heart of the City Center! Ideal for a family! (Apartment), Luxembourg (Luxembourg) deals', 'We Price Match', ' We Price Match ', ' Low rates • ', 'No booking fees', " • Find cheaper? We'll refund the difference! ", 'Search', 'Destination/property name:', 'Error: ', 'Please e

In [149]:
text = soup_room.find_all(text = True)
print (text)

['html', '\n', "\nYou know you could be getting paid to poke around in our code?\nWe're hiring designers and developers to work in Amsterdam:\nhttps://careers.booking.com/\n", '\n', ' wdot-802 ', '\n', '\ndocument.addEventListener(\'DOMContentLoaded\', function () {\n//c360 javascript tracker first iteration\n//sends a track request to c360 http tracker\n//in order to use it:\n//1. inline the c360Tracker.js in the page you need to use it\n//2. in your js file:\n//\n// var c360Tracker = B.require(\'c360Tracker\');\n// var event = {\n// action_name:"accommodation_checkout_confirmation_viewed",\n// action_version :"0.2.0",\n// content : { "transaction_id" : 123434},\n// user : { "BKNG_user_id": 123434}\n// };\n// c360Tracker.track(event);\nB.define(\'c360Tracker\', function () {\nvar enrichedContext = {};\nvar configuration = {\nvalidateInput: false\n};\nvar track = function (event) {\nif (event == null) {\nreturn "event object is null or empty";\n} else {\n//if (enrichedContext.userID !=

In [150]:
blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script']

In [153]:
output = []
for t in text:
    if t.parent.name not in blacklist:
        if t != '\n':
            output.append(t)
print(output)

[' #basiclayout, .basiclayout { margin: 0; } #special_actions { margin: 3px 15px 3px 0; } .ticker_space { margin-top: 3px !important; } #logo_no_globe_new_logo { top: 14px; } .b_msie_6 #top, .b_msie_6 body.header_reshuffle #top {height:61px !important;} .b_msie_6 #special_actions { margin: 3px 15px 3px 0; overflow:visible; } body.header_reshuffle #top { min-height: 50px !important; height: auto !important; } .nobg { background: #fff url("https://cf.bstatic.com/static/img/nobg_all_blue_iq/b700d9e3067c1186a3364012df4fe1c48ae6da44.png") repeat-x; background-position: 0 -50px; } ', "CAB Ap'art, Luxembourg – Updated 2022 Prices", '[if IE]> <script nonce="hIrvIGcQL1sIeUK"> document.createElement(\'time\'); </script> <![endif]', 'Skip to main content', '\nEUR\n', '\nChoose your currency.\nYour current currency is Euro\n', '\nChoose your language.\nYour current language is English (UK)\n', '\nSelect your language\n', '\nSuggested for you\n', '\nEnglish (US)\n', '\nPolski\n', '\nEspañol\n', '\n

In [130]:
info_pic = soup_room.find(class_ ="bh-photo-grid-item bh-photo-grid-thumb")

In [131]:
print(info_pic)

<a aria-label="" class="bh-photo-grid-item bh-photo-grid-thumb" data-et-click=" customGoal:NAREFBUONNBAVKfPBDOLHIbSbPTZEFO:2" data-et-mouseenter=" customGoal:NAREFBUONNBAVKfPBDOLHIbSbPTZEFO:1" data-id="137484643" data-preview-image-layout="thumbnail" data-preview-image-ranking="3" href="#" style="background-image: url(https://t-cf.bstatic.com/xdata/images/hotel/max300/137484643.jpg?k=947bda88848e9ae6f510cbb100e77e38f3145b1b828dfcfaaf87f46af37ed122&amp;o=&amp;hp=1);">
<img alt="" class="hide" src="https://t-cf.bstatic.com/xdata/images/hotel/max300/137484643.jpg?k=947bda88848e9ae6f510cbb100e77e38f3145b1b828dfcfaaf87f46af37ed122&amp;o=&amp;hp=1">
</img></a>


In [134]:
info_pic2 = info_pic.find('img')

In [136]:
info_pic2['src']

'https://t-cf.bstatic.com/xdata/images/hotel/max300/137484643.jpg?k=947bda88848e9ae6f510cbb100e77e38f3145b1b828dfcfaaf87f46af37ed122&o=&hp=1'